In [1]:
# Taken from: http://nbviewer.jupyter.org/github/fonnesbeck/Bios8366/tree/master/notebooks/styles/
from IPython.core.display import HTML
def css_styling():
    styles = open("styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

# Python for Data Analysis

The goal of this notebook is to bring my python skills up to speed with my R's skills. Let Hadley Wickham be praised; let's try to replicate that in Pandas

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('max_columns', 50)
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

pd.options.display.max_rows = 10
%matplotlib inline

In [3]:
from html5lib import *

# Data manipulation

Assume you have your data in tidy form. How to manipulate it? It's useful to remember Mr. Wickham's conceptualization of data analysis: there are six basic verbs for data manipulation. Filter, select, arrange, mutate, summarise and group by. Let's go through their python equivalent in order. 

## Filter and Select

Spoiler: __df.query()__ and __[]__

In [4]:
df = pd.DataFrame(np.random.randn(10, 3), columns=list('abc'))
df

,a,b,c
0,0.102522,1.358159,-0.096443
1,-0.255670,0.475740,-0.137979
2,-0.987709,0.641123,-0.834846
3,-0.844701,-1.894036,-1.039628
4,0.632934,0.490944,0.282190
5,0.786939,0.627002,-0.220635
6,0.053948,-0.851054,-0.395503
7,-0.316070,1.052777,1.130931
8,0.003496,0.344184,-0.105422
9,-0.936505,-0.530755,-0.582767


Using pandas slicing techniques you must remember the three following rules:
* Use [] for selecting columns
* Use .loc[row_lables, column_labels] for label-based indexing
* Use .iloc[row_positions, column_positions] for positional index

In [5]:
## a and b columns
df[['a','b']]

,a,b
0,0.102522,1.358159
1,-0.255670,0.475740
2,-0.987709,0.641123
3,-0.844701,-1.894036
4,0.632934,0.490944
5,0.786939,0.627002
6,0.053948,-0.851054
7,-0.316070,1.052777
8,0.003496,0.344184
9,-0.936505,-0.530755


In [6]:
# Choose rows with both and b being less than 1
df.query('a < 1 & b < 1')

,a,b,c
1,-0.255670,0.475740,-0.137979
2,-0.987709,0.641123,-0.834846
3,-0.844701,-1.894036,-1.039628
4,0.632934,0.490944,0.282190
5,0.786939,0.627002,-0.220635
6,0.053948,-0.851054,-0.395503
8,0.003496,0.344184,-0.105422
9,-0.936505,-0.530755,-0.582767


## Arrange

In [7]:
# Arrange data frame according to the values in column a, in descending order
df.sort_values('a', ascending = False)

,a,b,c
5,0.786939,0.627002,-0.220635
4,0.632934,0.490944,0.282190
0,0.102522,1.358159,-0.096443
6,0.053948,-0.851054,-0.395503
8,0.003496,0.344184,-0.105422
1,-0.255670,0.475740,-0.137979
7,-0.316070,1.052777,1.130931
3,-0.844701,-1.894036,-1.039628
9,-0.936505,-0.530755,-0.582767
2,-0.987709,0.641123,-0.834846


In [8]:
# Arrange data frame, in ascending order, first by a and then by b
df.sort_values(['a','b'])

,a,b,c
2,-0.987709,0.641123,-0.834846
9,-0.936505,-0.530755,-0.582767
3,-0.844701,-1.894036,-1.039628
7,-0.316070,1.052777,1.130931
1,-0.255670,0.475740,-0.137979
8,0.003496,0.344184,-0.105422
6,0.053948,-0.851054,-0.395503
0,0.102522,1.358159,-0.096443
4,0.632934,0.490944,0.282190
5,0.786939,0.627002,-0.220635


## Mutate

In [9]:
# Create variable d: sum of the other variables
df = df.assign(d = df['a'] + df['b'] + df['c'])
df

,a,b,c,d
0,0.102522,1.358159,-0.096443,1.364237
1,-0.255670,0.475740,-0.137979,0.082091
2,-0.987709,0.641123,-0.834846,-1.181432
3,-0.844701,-1.894036,-1.039628,-3.778365
4,0.632934,0.490944,0.282190,1.406067
5,0.786939,0.627002,-0.220635,1.193305
6,0.053948,-0.851054,-0.395503,-1.192609
7,-0.316070,1.052777,1.130931,1.867639
8,0.003496,0.344184,-0.105422,0.242258
9,-0.936505,-0.530755,-0.582767,-2.050027


## Rename

In [10]:
df = df.rename(columns = {'a': 'A'})
df

,A,b,c,d
0,0.102522,1.358159,-0.096443,1.364237
1,-0.255670,0.475740,-0.137979,0.082091
2,-0.987709,0.641123,-0.834846,-1.181432
3,-0.844701,-1.894036,-1.039628,-3.778365
4,0.632934,0.490944,0.282190,1.406067
5,0.786939,0.627002,-0.220635,1.193305
6,0.053948,-0.851054,-0.395503,-1.192609
7,-0.316070,1.052777,1.130931,1.867639
8,0.003496,0.344184,-0.105422,0.242258
9,-0.936505,-0.530755,-0.582767,-2.050027


## Split-Apply-Combine

The split-apply-combine is a famous strategy in data analysis. In R, you'd group_by() and then summarise with some measure for each group. In python: split according to some keys, apply some function to each group, and then combine the groups. 

In [11]:
df = pd.DataFrame({'Equipo' : ['Real Madrid', 'Real Madrid', 'Barcelona', 'Barcelona'], 'Goles' : [30, 40, 7, 8]})
df

,Equipo,Goles
0,Real Madrid,30
1,Real Madrid,40
2,Barcelona,7
3,Barcelona,8


In [12]:
# Let's group by the key X
by_team = df.groupby(['Equipo'])
by_team.get_group('Real Madrid')

,Equipo,Goles
0,Real Madrid,30
1,Real Madrid,40


In [13]:
# Let's summarise each team by the total number of goals they made, and the average goals they made
by_team.aggregate({'Goles' : [np.sum, np.mean]})

Goles      
              sum  mean
Equipo                 
Barcelona      15   7.5
Real Madrid    70  35.0

In [14]:
# Group, choose by what variable you want to summarize, and then say how you wanna summarise it. 
by_team['Goles'].aggregate({'Avg_Goals': np.mean ,
                            'Total_Goals': np.sum ,
                            'Number_matches' : np.size})

,Avg_Goals,Number_matches,Total_Goals
Equipo,,,
Barcelona,7.5,2,15
Real Madrid,35.0,2,70


To do the equivalent of pipe (%>%) in R, and avoid horizontally nesting functions:

In [59]:
(df
 .groupby(['Equipo'])
         .Goles
         .aggregate({'Avg_Goals': np.mean ,
                            'Total_Goals': np.sum ,
                            'Number_matches' : np.size}))

,Number_matches,Avg_Goals,Total_Goals
Equipo,,,
Barcelona,2,7.5,15
Real Madrid,2,35.0,70


### Useful trick to create individual dummy variables

In [16]:
df['Good_team'] = np.where(df['Equipo']=='Real Madrid','Yes!','No')

In [17]:
# .ix combines loc and iloc
df['Good_team'] = 0
df.ix[df['Equipo'] == 'Real Madrid', 'Good_team'] = 'Yes!'
df

,Equipo,Goles,Good_team
0,Real Madrid,30,Yes!
1,Real Madrid,40,Yes!
2,Barcelona,7,0
3,Barcelona,8,0


# Tidying Data

Remember: in a tidy data frame, every row is an observation and every column is a variable. Rarely, read data comes in such format. Let's see some of the common problems: a contains multiple variables, in which case we'd use __spread()__ in R ; there are many value-column variables, in which case we'd use __gather()__ in R. 

An example: when a column contains many variables, we say the data is stacked. Spread()

In [69]:
import pandas.util.testing as tm; tm.N = 3
def unpivot(frame):
    N, K = frame.shape
    data = {'value' : frame.values.ravel('F'),
            'variable' : np.asarray(frame.columns).repeat(N),
            'date' : np.tile(np.asarray(frame.index), K)}
    return pd.DataFrame(data, columns=['date', 'variable', 'value'])
df = unpivot(tm.makeTimeDataFrame())
df

,date,variable,value
0,2000-01-03,A,-0.051888
1,2000-01-04,A,-2.089794
2,2000-01-05,A,1.484783
3,2000-01-03,B,-1.719927
4,2000-01-04,B,0.079183
...,...,...,...
7,2000-01-04,C,0.358552
8,2000-01-05,C,-0.461155
9,2000-01-03,D,-0.351592
10,2000-01-04,D,-0.143884


The column 'variable' contains many real keys. Let's unstack it: in python, use pivot. The operation is parametrized by the columns that are not value-columns, by the colum that takes many variables, and by the column that takes the value of the respective keys.

In [70]:
df2 = df.pivot_table(index='date', columns='variable', values='value').reset_index()
# parametrized by which column contains the different keys and which key contains their respective values

variable,date,A,B,C,D
0,2000-01-03,-0.051888,-1.719927,0.218590,-0.351592
1,2000-01-04,-2.089794,0.079183,0.358552,-0.143884
2,2000-01-05,1.484783,0.608405,-0.461155,-0.015355


Now, to do the opposite use __melt()__. That is, if you have value-columns and need to put them in one variable, what one would do with __spread()__ in R. The operation is parametrized by the columns that are tidy, the value-columns and the name of the new variable that will contain the cells you are stacking on one variable

In [71]:
pd.melt(df2, id_vars = ['date'], value_vars = ['A','B','C','D'], value_name = 'value' )

,date,variable,value
0,2000-01-03,A,-0.051888
1,2000-01-04,A,-2.089794
2,2000-01-05,A,1.484783
3,2000-01-03,B,-1.719927
4,2000-01-04,B,0.079183
...,...,...,...
7,2000-01-04,C,0.358552
8,2000-01-05,C,-0.461155
9,2000-01-03,D,-0.351592
10,2000-01-04,D,-0.143884


## Relational Database operations

In [3]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3'],
                    'C': ['C0', 'C1', 'C2', 'C3'],
                    'D': ['D0', 'D1', 'D2', 'D3']},
                    index=[0, 1, 2, 3])
df1

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


In [7]:
df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                    'B': ['B4', 'B5', 'B6', 'B7'],
                    'C': ['C4', 'C5', 'C6', 'C7'],
                    'D': ['D4', 'D5', 'D6', 'D7']},
                     index=[4, 5, 6, 7])
df2

,A,B,C,D
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


In [10]:
df3 = pd.DataFrame({'B': ['B2', 'B3', 'B6', 'B7'],
                    'D': ['D2', 'D3', 'D6', 'D7'],
                    'F': ['F2', 'F3', 'F6', 'F7']},
                     index=[2, 3, 6, 7])
df3

,B,D,F
2,B2,D2,F2
3,B3,D3,F3
6,B6,D6,F6
7,B7,D7,F7


If you just wanna stack one over the other, what you'd in R with rbind(), then use concatenate. Note: they must have the same columns

In [12]:
frames = [df1, df2]
pd.concat(frames)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


If, on the other hand, you want to merge, then use pd.merge. Its how argument will indicate what type of SQL operation you want to perform, i.e., inner, outer, left or right. If you're not operating on indexes, use the argument on and pass a list of the columns that uniquely identify the observations. 

In [14]:
pd.merge(df1, df3, how = 'left', left_index = True , right_index = True)

,A,B_x,C,D_x,B_y,D_y,F
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3


In [15]:
pd.merge(df1, df3, how = 'inner', left_index = True, right_index = True)

,A,B_x,C,D_x,B_y,D_y,F
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3
